## Device Setup

In [1]:
!nvidia-smi

Sun Dec 21 08:03:34 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   51C    P0              46W / 300W |      4MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
print(f"Current PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

Current PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA version: 12.8


## Passage dataset

***
10 random seeds: range(20, 30)
for data creation for each type of spammer

invoke bt

get bt accuracy (+- std dev), wacc and tau

save in results/spammer_type/bt.csv
***

In [4]:
import sys
sys.path.insert(0, "../")
sys.path.insert(1, "../../")

from spammer_types import *
from util import *
import opt_fair
from distribution_utils import crowd_bt_dist, logistic_preference_dist, comparisons_to_df, safe_kendalltau, to_numpy
from metrics import compute_acc, compute_weighted_acc
import choix

### Get the original df of passage dataset

In [5]:
df_path = "../../real_data/passage/data/passage_cleaned.csv"

In [6]:
import pandas as pd
df = pd.read_csv(df_path)
def sort_df(df, column_name):
        # Sort by a specific column (replace 'column_name' with your column)
        df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

        return df_sorted
df = sort_df(df, 'performer')
df[['left', 'right', 'label', 'performer']].head()

,left,right,label,performer
0,been wicked. They believed that the end of the...,lichen Sect Content Linking Artid A snake coil...,lichen Sect Content Linking Artid A snake coil...,5
20,"school, you noticed that all the clocks read a...",man in the seat with me and the two women acro...,man in the seat with me and the two women acro...,5
19,foreign animals or plants may be taken into th...,adhere to the wall. Using sizing that has been...,foreign animals or plants may be taken into th...,5
18,many pennies did each child get Which computat...,the picture on its cover. Write two or three s...,many pennies did each child get Which computat...,5
17,direction of the Sun. The length in meters and...,"the ground going into the cave. Oh, my he said...","the ground going into the cave. Oh, my he said...",5


In [7]:
percents = [10, 20, 40, 60, 80]
# percents = [10]

In [8]:
import pickle

with open("../../real_data/passage/data/PassageDF.pickle", "rb") as handle:
    df_passage = pickle.load(handle)
df_passage

,label,score
0,"a star. Our planet, Earth, orbits, or circles,...",1
1,"Adam, We did not have plastic toys. I played w...",1
2,Who said the little owl. Who wants to hunt wit...,1
3,dead leaf. This is a mole. Moles burrow underg...,1
4,ereaddatagradepsenvironcomp.html Environment r...,1
...,...,...
467,work over the summer on any changes they wish ...,12
468,between January and December plunged the Unite...,12
469,into a newly opened bank account. I was amazed...,12
470,"occurring phenomenon, manmade by products are ...",12


In [9]:
size = len(df_passage)
print(size)
classes = [0] * size
# for faceage it would be classes = df_passage['gender']

472


In [10]:
gt_df = df_passage

### Addition of Random Guessors

In [11]:
spammer_type = "random"

In [12]:
csv_file = f"results/{spammer_type}/bt.csv"

In [13]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [14]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BT_acc_mean", "BT_acc_std",
    "BT_wacc_mean", "BT_wacc_std",
    "BT_tau_mean", "BT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [15]:
for percent in percents:
    # initialize metrics
    BT_accs, BT_waccs, BT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_random_spammer(df, percent, seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        all_pc_faceage = opt_fair._pc_without_reviewers(PC_faceage)
        df_faceage = df_passage
        
        try:
            bt_scores = choix.opt_pairwise(size, all_pc_faceage, alpha=0, method='Newton-CG', initial_params=None, max_iter=None, tol=1e-05)
            bt_scores = to_numpy(bt_scores)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
            if bt_tau < 0:
                bt_scores = -bt_scores
            bt_acc = compute_acc(df_faceage, bt_scores, device)
            bt_wacc = compute_weighted_acc(df_faceage, bt_scores, device)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
        except Exception as e:
            print(f"BT failed: {e}")
            continue
        BT_accs.append(bt_acc)
        BT_waccs.append(bt_wacc)
        BT_taus.append(bt_tau)
    
    row = [
        percent,
        np.mean(BT_accs), np.std(BT_accs),
        np.mean(BT_waccs), np.std(BT_waccs),
        np.mean(BT_taus), np.std(BT_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BT_accs):.4f} ± {np.std(BT_accs):.4f} | "
    f"WAcc: {np.mean(BT_waccs):.4f} ± {np.std(BT_waccs):.4f} | "
    f"Tau: {np.mean(BT_taus):.4f} ± {np.std(BT_taus):.4f}"
)

Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
BT | Percent: 10 |Acc: 0.6780 ± 0.0028 | WAcc: 0.7403 ± 0.0034 | Tau: 0.3361 ± 0.0053
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
BT | Percent: 20 |Acc: 0.6770 ± 0.0040 | WAcc: 0.7384 ± 0.0054 | Tau: 0.3343 ± 0.0076
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
BT | Percent: 40 |

### Addition of Anti-Personas

In [16]:
spammer_type = "anti"

In [17]:
csv_file = f"results/{spammer_type}/bt.csv"

In [18]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [19]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BT_acc_mean", "BT_acc_std",
    "BT_wacc_mean", "BT_wacc_std",
    "BT_tau_mean", "BT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [20]:
for percent in percents:
    # initialize metrics
    BT_accs, BT_waccs, BT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_anti_personas(df, percent, seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        all_pc_faceage = opt_fair._pc_without_reviewers(PC_faceage)
        df_faceage = df_passage
        
        try:
            bt_scores = choix.opt_pairwise(size, all_pc_faceage, alpha=0, method='Newton-CG', initial_params=None, max_iter=None, tol=1e-05)
            bt_scores = to_numpy(bt_scores)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
            if bt_tau < 0:
                bt_scores = -bt_scores
            bt_acc = compute_acc(df_faceage, bt_scores, device)
            bt_wacc = compute_weighted_acc(df_faceage, bt_scores, device)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
        except Exception as e:
            print(f"BT failed: {e}")
            continue
        BT_accs.append(bt_acc)
        BT_waccs.append(bt_wacc)
        BT_taus.append(bt_tau)
    
    row = [
        percent,
        np.mean(BT_accs), np.std(BT_accs),
        np.mean(BT_waccs), np.std(BT_waccs),
        np.mean(BT_taus), np.std(BT_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BT_accs):.4f} ± {np.std(BT_accs):.4f} | "
    f"WAcc: {np.mean(BT_waccs):.4f} ± {np.std(BT_waccs):.4f} | "
    f"Tau: {np.mean(BT_taus):.4f} ± {np.std(BT_taus):.4f}"
)

Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
BT | Percent: 10 |Acc: 0.6778 ± 0.0004 | WAcc: 0.7401 ± 0.0006 | Tau: 0.3358 ± 0.0008
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
BT | Percent: 20 |Acc: 0.6739 ± 0.0010 | WAcc: 0.7323 ± 0.0016 | Tau: 0.3284 ± 0.0019
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
BT | Percent: 40 |

### Addition of Left Position Biased Spammers

In [21]:
spammer_type = "left"

In [22]:
csv_file = f"results/{spammer_type}/bt.csv"

In [23]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [24]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BT_acc_mean", "BT_acc_std",
    "BT_wacc_mean", "BT_wacc_std",
    "BT_tau_mean", "BT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [25]:
for percent in percents:
    # initialize metrics
    BT_accs, BT_waccs, BT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent, position_bias="left", seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        all_pc_faceage = opt_fair._pc_without_reviewers(PC_faceage)
        df_faceage = df_passage
        
        try:
            bt_scores = choix.opt_pairwise(size, all_pc_faceage, alpha=0, method='Newton-CG', initial_params=None, max_iter=None, tol=1e-05)
            bt_scores = to_numpy(bt_scores)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
            if bt_tau < 0:
                bt_scores = -bt_scores
            bt_acc = compute_acc(df_faceage, bt_scores, device)
            bt_wacc = compute_weighted_acc(df_faceage, bt_scores, device)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
        except Exception as e:
            print(f"BT failed: {e}")
            continue
        BT_accs.append(bt_acc)
        BT_waccs.append(bt_wacc)
        BT_taus.append(bt_tau)
    
    row = [
        percent,
        np.mean(BT_accs), np.std(BT_accs),
        np.mean(BT_waccs), np.std(BT_waccs),
        np.mean(BT_taus), np.std(BT_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BT_accs):.4f} ± {np.std(BT_accs):.4f} | "
    f"WAcc: {np.mean(BT_waccs):.4f} ± {np.std(BT_waccs):.4f} | "
    f"Tau: {np.mean(BT_taus):.4f} ± {np.std(BT_taus):.4f}"
)

Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
BT | Percent: 10 |Acc: 0.6676 ± 0.0004 | WAcc: 0.7306 ± 0.0007 | Tau: 0.3166 ± 0.0008
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
BT | Percent: 20 |Acc: 0.6625 ± 0.0015 | WAcc: 0.7212 ± 0.0021 | Tau: 0.3070 ± 0.0029
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
BT | Percent: 40 |

### Addition of Right Position Biased Spammers

In [26]:
spammer_type = "right"

In [27]:
csv_file = f"results/{spammer_type}/bt.csv"

In [28]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [29]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BT_acc_mean", "BT_acc_std",
    "BT_wacc_mean", "BT_wacc_std",
    "BT_tau_mean", "BT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
for percent in percents:
    # initialize metrics
    BT_accs, BT_waccs, BT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent, position_bias="right", seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        all_pc_faceage = opt_fair._pc_without_reviewers(PC_faceage)
        df_faceage = df_passage
        
        try:
            bt_scores = choix.opt_pairwise(size, all_pc_faceage, alpha=0, method='Newton-CG', initial_params=None, max_iter=None, tol=1e-05)
            bt_scores = to_numpy(bt_scores)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
            if bt_tau < 0:
                bt_scores = -bt_scores
            bt_acc = compute_acc(df_faceage, bt_scores, device)
            bt_wacc = compute_weighted_acc(df_faceage, bt_scores, device)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
        except Exception as e:
            print(f"BT failed: {e}")
            continue
        BT_accs.append(bt_acc)
        BT_waccs.append(bt_wacc)
        BT_taus.append(bt_tau)
    
    row = [
        percent,
        np.mean(BT_accs), np.std(BT_accs),
        np.mean(BT_waccs), np.std(BT_waccs),
        np.mean(BT_taus), np.std(BT_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BT_accs):.4f} ± {np.std(BT_accs):.4f} | "
    f"WAcc: {np.mean(BT_waccs):.4f} ± {np.std(BT_waccs):.4f} | "
    f"Tau: {np.mean(BT_taus):.4f} ± {np.std(BT_taus):.4f}"
)

Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
Unique performers: 686
686
BT | Percent: 10 |Acc: 0.6844 ± 0.0003 | WAcc: 0.7455 ± 0.0003 | Tau: 0.3483 ± 0.0006
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
Unique performers: 748
748
BT | Percent: 20 |Acc: 0.6807 ± 0.0012 | WAcc: 0.7424 ± 0.0015 | Tau: 0.3412 ± 0.0023
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873
Unique performers: 873
873


### Addition of Equal proportion of all spammers

In [ ]:
spammer_type = "equal"

In [ ]:
csv_file = f"results/{spammer_type}/bt.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "BT_acc_mean", "BT_acc_std",
    "BT_wacc_mean", "BT_wacc_std",
    "BT_tau_mean", "BT_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
for percent in percents:
    # initialize metrics
    BT_accs, BT_waccs, BT_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_equal_proportion_of_all_spammers(df, percent, seed=sd)
        PC_faceage = df_to_pickle(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        all_pc_faceage = opt_fair._pc_without_reviewers(PC_faceage)
        df_faceage = df_passage
        
        try:
            bt_scores = choix.opt_pairwise(size, all_pc_faceage, alpha=0, method='Newton-CG', initial_params=None, max_iter=None, tol=1e-05)
            bt_scores = to_numpy(bt_scores)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
            if bt_tau < 0:
                bt_scores = -bt_scores
            bt_acc = compute_acc(df_faceage, bt_scores, device)
            bt_wacc = compute_weighted_acc(df_faceage, bt_scores, device)
            bt_tau = safe_kendalltau(bt_scores, df_faceage['score'].to_numpy())
        except Exception as e:
            print(f"BT failed: {e}")
            continue
        BT_accs.append(bt_acc)
        BT_waccs.append(bt_wacc)
        BT_taus.append(bt_tau)
    
    row = [
        percent,
        np.mean(BT_accs), np.std(BT_accs),
        np.mean(BT_waccs), np.std(BT_waccs),
        np.mean(BT_taus), np.std(BT_taus)
    ]
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"BT | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(BT_accs):.4f} ± {np.std(BT_accs):.4f} | "
    f"WAcc: {np.mean(BT_waccs):.4f} ± {np.std(BT_waccs):.4f} | "
    f"Tau: {np.mean(BT_taus):.4f} ± {np.std(BT_taus):.4f}"
)